In [2]:
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
#= Separa el experimento en σ segmentos, convierte a voltaje, obtiene una matriz filtrada 
de MUA, calcula variables para caracterizar cada segmento =#

# ~~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ constantes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
limupper = 0.4; # GB, para la segmentaci'on del archivo completo
ChPercent = 15; # % porcentaje del canal que se tolera saturado
FrPercent = 20; # % porcentaje del frame que se tolera saturado
ΔT = 400; # ms  para la variacion temporal
window = 5; # ms para la deteccion de eventos
bit = 1; # ms deslizamiento de la ventana
distance = 0.3; # ms distancia maxima entre eventos para ser descartados
cte = 4; # 4 / sqrt( 3 ), for the donoho threshold
thr = -60; # μV umbral estatico
nM = 18;
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ paquetes ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
using JLD
using StatsBase
using Plots
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
PATHFunctions = "/home/isabel/Dropbox/git-repos/Spinal-Cord-HDMEA";
push!( LOAD_PATH, PATHFunctions );
using MeaTools
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
FILEBRW = "/run/media/isabel/Data/Hipocampo/Espontanea/01-06-2022/BRWs/Phase_10.brw";
Variables, FILEPATHS = GetVarsHDF5( FILEBRW ); # Variables generales del archivo
PATHS = load( FILEPATHS )[ "PATHS" ];

FILEVARS = joinpath( PATHS[ "PATHINFO" ], "Variables.jld" );
PATHVOLTAGE = joinpath( PATHS[ "PATHMAIN" ], "Voltage" ); mkpath( PATHVOLTAGE );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
σ = ChunkSizeSpace( Variables, limupper ); # N'umero de segmentos a cortar
nChs = Variables[ "nChs" ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Wn = Array{ Float64 }( undef, nChs, nM*σ );
@time for n = 1:σ
    n0s = length( string( σ ) );
    BINname = joinpath( PATHVOLTAGE, string( "BIN", lpad( n, n0s, "0" ), ".jld" ) );
    BINRaw = OneSegment( Variables, n, σ ); # Extrae el bin crudo
    BINRaw = Digital2Analogue( Variables, BINRaw ); # conversi'on a voltaje
    BINRaw, DiscardedFrames, DiscardedChannels, Frs4Repair, Chs4Repair, 
        PossibleEarth = DeSatMinMax( Variables, BINRaw, ChPercent, FrPercent );
    BINRaw[ PossibleEarth, : ] .= 0;
    W, Events, BINFilt = AllMetrics( 
        Variables, BINRaw, ΔT, window, bit, distance, cte, thr );
    Wn[ :, collect( n:σ:nM*σ ) ] = W;
    save( BINname, 
            "data", Float16.( BINRaw ),
            "filtered", Float16.( BINFilt ),
            "NOFr", DiscardedFrames,
            "NOChs", DiscardedChannels,
            "FrsRep", Frs4Repair,
            "ChsRep", Chs4Repair,
            "Discarted", PossibleEarth,
            "Pesos", W,
            "Eventos", Events
            );
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
PATHFIGURES = joinpath( PATHS[ "PATHMAIN" ], "Figures" ); 
PATHWFIGS = joinpath( PATHFIGURES, "WFIGS" ); mkpath( PATHWFIGS );
PATHFIGSG = joinpath( PATHFIGURES, "General" ); mkpath( PATHFIGSG ); 
FILEWs = joinpath( PATHS[ "PATHINFO" ], "AllWs.jld" );
GeneralPlots = Array{ Plots.Plot{ Plots.GRBackend } }( undef, nM );
W0NTotal = Array{ Float64 }( undef, nChs, nM );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

You are now working on the new main path: /run/media/LBitn/Data/Hipocampo/Espontanea/01-06-2022/BRWs/Phase_10
With the file: 
Phase_10.brw : BRW-File Level3 - 3Brain RaW data file for high resolution MEA platform, HDF5-format - Created with BrainWave v.4.5.8180.23837 on date Wednesday, June 1, 2022 HDF5 file size: 25.7256636640625 GB
65 segments of 0.3957794409855769 GB each one


┌ Info: Precompiling CSDATools [top-level]
└ @ Base loading.jl:1423


In [ ]:
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# manualmente ajustar el parametro mid
n = 1;
mid = 0.91; # <---------------------------------------------------------------------------- #
P, W0N = WGeneral( Wn, n, σ, mid ); GeneralPlots[ n ] = P; W0NTotal[ :, n ] = W0N;
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

In [169]:
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
save( FILEWs, "Wn", Wn, "W0NTotal", W0NTotal );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
PF = plot( GeneralPlots..., layout = ( 6, 3 ), cbar = :none, wsize = ( 800, 1200 ) );
savefig( PF, joinpath( PATHFIGSG, "AllWs.png" ) ); 
savefig( PF, joinpath( PATHFIGSG, "AllWs.svg" ) );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

In [ ]:
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
save( joinpath( PATHS[ "PATHINFO" ], "GeneralPlots.jld" ), "GeneralPlots", GeneralPlots ); 
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #